# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [41]:
!pip install chromadb langchain langchain_community langchain_openai

In [42]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())



In [45]:
llm = OpenAI(temperature=0)

In [46]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "hamlet.txt")

In [48]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="hamlet", persist_directory="./chroma_db")

In [49]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [50]:
from langchain_community.document_loaders import WebBaseLoader

In [51]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [54]:
docs = loader.load()
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
persist_directory="./chroma_db"
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff", persist_directory=persist_directory)
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [55]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [57]:
tools = [
    Tool(
        name="hamlet QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent hamlet address. Input should be a fully formed question.",
        return_direct=True,  # Add return_direct=True
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,  # Add return_direct=True
    ),
]

In [58]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [59]:
agent.invoke(
    "What does the Ghost say about his brother’s role in his death?")



> Entering new AgentExecutor chain...
 I should use the hamlet QA System to answer this question since it is related to the play Hamlet.
Action: hamlet QA System
Action Input: "What does the Ghost say about his brother’s role in his death?"
Observation:  The Ghost reveals that his brother, now the current king, is the one who murdered him.


> Finished chain.


{'input': 'What does the Ghost say about his brother’s role in his death?',
 'output': ' The Ghost reveals that his brother, now the current king, is the one who murdered him.'}

In [60]:
agent.invoke("How does Hamlet initially react to the news of his father's murder?")



> Entering new AgentExecutor chain...
 Hamlet's initial reaction is important to understanding his character and motivations.
Action: hamlet QA System
Action Input: "How does Hamlet initially react to the news of his father's murder?"
Observation:  Hamlet is shocked and overwhelmed by the news of his father's murder. He exclaims "O God!" and "Murder!" upon hearing it from the Ghost. He also immediately expresses a desire for revenge, saying "Haste me to know't, that I, with wings as swift/As meditation or the thoughts of love,/May sweep to my revenge."


> Finished chain.


{'input': "How does Hamlet initially react to the news of his father's murder?",
 'output': ' Hamlet is shocked and overwhelmed by the news of his father\'s murder. He exclaims "O God!" and "Murder!" upon hearing it from the Ghost. He also immediately expresses a desire for revenge, saying "Haste me to know\'t, that I, with wings as swift/As meditation or the thoughts of love,/May sweep to my revenge."'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [61]:
tools = [
    Tool(
        name="hamlet QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent hamlet address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [62]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [63]:
agent.invoke(
    "How does Hamlet respond to the Ghost’s call for revenge?"
)



> Entering new AgentExecutor chain...
 Hamlet is a QA System, so it should be able to provide a detailed answer based on the text of the play.
Action: hamlet QA System
Action Input: "How does Hamlet respond to the Ghost’s call for revenge?"
Observation:  Hamlet responds with a sense of urgency and determination, stating that he will swiftly seek revenge. He also expresses his prophetic soul and realization that his uncle is the one who murdered his father.


> Finished chain.


{'input': 'How does Hamlet respond to the Ghost’s call for revenge?',
 'output': ' Hamlet responds with a sense of urgency and determination, stating that he will swiftly seek revenge. He also expresses his prophetic soul and realization that his uncle is the one who murdered his father.'}

In [64]:
agent.invoke("What advice does the Ghost give Hamlet regarding his mother, Queen Gertrude?")



> Entering new AgentExecutor chain...
 The Ghost is a supernatural being, so its advice may not be reliable.
Action: hamlet QA System
Action Input: "What advice does the Ghost give Hamlet regarding his mother, Queen Gertrude?"
Observation:  The Ghost advises Hamlet to speak to his mother and try to intervene between her and her inner turmoil.


> Finished chain.


{'input': 'What advice does the Ghost give Hamlet regarding his mother, Queen Gertrude?',
 'output': ' The Ghost advises Hamlet to speak to his mother and try to intervene between her and her inner turmoil.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [65]:
tools = [
    Tool(
        name="hamlet QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent hamlet address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [66]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [67]:
agent.invoke(
    "Why do Horatio and Marcellus become concerned about Hamlet’s state of mind?"
)



> Entering new AgentExecutor chain...
 They must have noticed some unusual behavior from Hamlet.
Action: hamlet QA System
Action Input: "Why do Horatio and Marcellus become concerned about Hamlet’s state of mind?"
Observation:  Horatio and Marcellus become concerned about Hamlet's state of mind because he is speaking to a ghost and seems to be in a state of shock and confusion. They are worried about his mental well-being and want to make sure he is okay.
Thought: This could be a sign of something more serious going on with Hamlet.
Action: hamlet QA System
Action Input: "What could be the cause of Hamlet's unusual behavior?"
Observation:  The cause of Hamlet's unusual behavior could be his grief over his father's death and his mother's hasty remarriage to his uncle, as well as his suspicions about his uncle's involvement in his father's death.
Thought: It seems like there are a lot of factors contributing to Hamlet's state of mind.
Action: hamlet QA System
Action Input: "What other f

{'input': 'Why do Horatio and Marcellus become concerned about Hamlet’s state of mind?',
 'output': "Horatio and Marcellus become concerned about Hamlet's state of mind because of his unusual behavior, which could be caused by his grief, anger, and suspicions towards his uncle, as well as the pressure and expectations placed on him as the prince of Denmark. To help Hamlet, they can offer support and assistance in his quest for revenge and justice."}

In [68]:
agent.invoke(
    "How does Shakespeare use the Ghost as a symbol in this scene?"
)



> Entering new AgentExecutor chain...
 I should use the hamlet QA System to answer this question.
Action: hamlet QA System
Action Input: How does Shakespeare use the Ghost as a symbol in this scene?
Observation:  In this scene, Shakespeare uses the Ghost as a symbol of the past and the truth. The Ghost represents the past because it is the spirit of Hamlet's father, who has been dead for some time. It also represents the truth because it reveals to Hamlet the true nature of his father's death and urges him to seek revenge. The Ghost's appearance also symbolizes the disruption and corruption in the state of Denmark, as it is a sign of the unrest caused by the murder of the king.
Thought: I now know the final answer.
Final Answer: In this scene, Shakespeare uses the Ghost as a symbol of the past, the truth, and the disruption and corruption in the state of Denmark.

> Finished chain.


{'input': 'How does Shakespeare use the Ghost as a symbol in this scene?',
 'output': 'In this scene, Shakespeare uses the Ghost as a symbol of the past, the truth, and the disruption and corruption in the state of Denmark.'}